<a href="https://colab.research.google.com/github/DrFrank25/Syndecan_4-Ag73/blob/main/gmx_MMPBSA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Binding Free Energy Calculation (MM-PBSA + Entropy)**

In [ ]:
# ==========================================
# 🚀 Binding Free Energy Calculation (MM-PBSA + Entropy) in Colab
# ==========================================
# -- Import Libraris ----
import os
import subprocess
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# ===============================
# 1) Install GROMACS + gmx_MMPBSA
# ===============================
!mamba install -y -c conda-forge gromacs gmx-mmpbsa


In [ ]:
# ===============================
# 2) Upload the required input files
# ===============================
from google.colab import files

print("📂 Please upload: traj.xtc, topol.tpr, index.ndx, and mmpbsa.in")
uploaded = files.upload()

In [ ]:
# ===============================
# 3) Run MM-PBSA (Enthalpy contribution)
# ===============================
!mkdir -p MMPBSA

cmd_mmpbsa = """
g_mmpbsa -f traj.xtc -s topol.tpr -n index.ndx -i mmpbsa.in \
         -o MMPBSA/FINAL_RESULTS_MMPBSA.dat -nogui
"""
subprocess.run(cmd_mmpbsa, shell=True, check=True)

print("✅ MM-PBSA calculation completed.")


In [ ]:
# ===============================
# 4) Entropy calculation (Quasi-harmonic Analysis)
# ===============================
# --- Covariance matrix ---
cmd_covar = "echo 1 1 | gmx covar -s topol.tpr -f traj.xtc -o eigenval.xvg -v eigenvec.trr"
subprocess.run(cmd_covar, shell=True, check=True)

# --- Eigenvector analysis with entropy estimation ---
cmd_anaeig = "echo 1 | gmx anaeig -f traj.xtc -v eigenvec.trr -entropy entropy.xvg"
subprocess.run(cmd_anaeig, shell=True, check=True)

print("✅ Entropy calculation (QHA) completed.")

In [ ]:
# ===============================
# 5) Parse results
# ===============================

# --- Parse MM-PBSA results ---
mm_file = "MMPBSA/FINAL_RESULTS_MMPBSA.dat"
mm_results = {}
with open(mm_file) as f:
    for line in f:
        if line.strip() and not line.startswith("#"):
            parts = line.split()
            if len(parts) >= 2:
                try:
                    mm_results[parts[0]] = float(parts[1])
                except:
                    pass

# --- Parse entropy results ---
entropy_file = "entropy.xvg"
TDeltaS = None
with open(entropy_file) as f:
    for line in f:
        if not line.startswith(("@", "#")):
            vals = line.split()
            if len(vals) >= 2:
                try:
                    TDeltaS = float(vals[1])
                except:
                    pass
TDeltaS = abs(TDeltaS) if TDeltaS else 0.0

In [ ]:
# ===============================
# 6) Compute the final binding free energy
# ===============================
DeltaH = mm_results.get("DeltaTOTAL", 0.0)
DeltaG_total = DeltaH - TDeltaS


In [ ]:
# ===============================
# 7) Save final results
# ===============================
out = {
    "ΔE_vdw": mm_results.get("DeltaE_vdw", None),
    "ΔE_ele": mm_results.get("DeltaE_ele", None),
    "ΔG_pol": mm_results.get("DeltaG_pol", None),
    "ΔG_np":  mm_results.get("DeltaG_np", None),
    "ΔH (MM-PBSA)": DeltaH,
    "-TΔS (Entropy)": -TDeltaS,
    "ΔG_total (Binding Free Energy)": DeltaG_total
}

df = pd.DataFrame([out])
df.to_csv("Final_Binding_Free_Energy.csv", index=False)
print("✅ Final results saved as Final_Binding_Free_Energy.csv")

df

In [ ]:
# ===============================
# 8) Visualization - Energy Decomposition
# ===============================
labels = list(out.keys())
values = list(out.values())

plt.figure(figsize=(10,6))
bars = plt.bar(labels, values, color="skyblue", edgecolor="black")
plt.xticks(rotation=45, ha="right")
plt.ylabel("Energy (kJ/mol)")
plt.title("Binding Free Energy Decomposition (MM-PBSA + Entropy)")

# Annotate values on top of bars
for bar in bars:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2, height,
             f"{height:.2f}", ha='center', va='bottom', fontsize=8)

plt.tight_layout()
plt.savefig("Energy_Decomposition.png", dpi=1200)
plt.show()

print("📊 Plot saved as Energy_Decomposition.png")